# Client-Side Code

In [6]:
import color_classify
import joblib
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing import image as kimage
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from PIL import Image

### Inference Model

In [7]:
# Half of the Inference Model Stored on Client Side
class ClientResNet(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super(ClientResNet, self).__init__(*args, **kwargs)
        
        # Load ResNet50 pre-trained model without top (fully connected) layers
        resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        resnet_base.trainable = False
        
        # Get the output of the first three convolutional layers
        middle_layer = resnet_base.get_layer('conv3_block4_out')
        self.seq0 = Model(inputs=resnet_base.input, outputs=middle_layer.output)
        
        resnet_base = None

        # Get cluster centers for cluster-based differential privacy
        self.color_centers = {}
        for i in range(0, 30):
            self.color_centers[i] = np.load(f"../polyvore_outfits/noise_embeddings/color_emb_{i}.npy")

        self.gmm_model = joblib.load('../polyvore_outfits/gmm_color_model.pkl')

    # Convert image into right dimensions
    def preprocess_image(self, img):
        img_array = kimage.img_to_array(img)
        expand_img = np.expand_dims(img_array, axis=0)
        return preprocess_input(expand_img)
    
    # Add laplacian noise to embedding
    def add_laplacian_noise(self, emb, noise_scale):
        laplace_noise = np.random.laplace(0.0, noise_scale, size=emb.shape)
        emb += laplace_noise
        return emb
    
    # Add laplacian noise to embedding that is sign-controlled
    def add_cluster_based_noise(self, emb, item_rgb, noise_scale):
        # Get probabilities that it belongs to a certain cluster
        cluster_idx = self.gmm_model.predict(np.array(item_rgb).reshape(1, -1))[0]

        # Add controlled-random noise
        laplacian_noise = np.random.laplace(0.0, noise_scale, size=emb.shape)

        # Calculate distances after adding/subtracting the Laplacian noise
        distance_after_add = np.abs(emb + laplacian_noise - self.color_centers[cluster_idx])
        distance_after_sub = np.abs(emb - laplacian_noise - self.color_centers[cluster_idx])

        # Compare distances to determine direction of noise addition
        add_mask = distance_after_add > distance_after_sub
        subtract_mask = ~add_mask

        # Apply noise according to the determined direction
        laplacian_noise *= add_mask.astype(float) - subtract_mask.astype(float)

        return emb + laplacian_noise
        
    # Forward pass of the client model
    def predict(self, image_path, laplace_noise=False, cluster_noise=False, noise_scale=0.35):
        # Open and process the image
        img = Image.open(image_path)
        img = img.resize((224, 224)).convert("RGB")

        preprocessed_img = self.preprocess_image(img)
        img.close()

        # Feed through ResNet
        emb = self.seq0(preprocessed_img)

        # Apply noise
        if laplace_noise:
            emb = self.add_laplacian_noise(emb, noise_scale)
        elif cluster_noise:
            item_rgb = color_classify.get_color(image_path)
            emb = self.add_cluster_based_noise(emb, item_rgb, noise_scale)

        return emb
    
    # Needed for subclass
    def call(self, inputs):
        return self.seq0(inputs)

# Instantiate Model
ClientModel = ClientResNet()

In [8]:
# Perform Inference on image path
image_path = "jeans.png"

# Predict without noise
client_embeddings = ClientModel.predict(image_path)
np.save("client_embeddings.npy", client_embeddings)

# Predict with Laplace noise
noise_embeddings = ClientModel.predict(image_path, False, True)
np.save("noise_embeddings.npy", noise_embeddings)

# Predict with CLuster noise
cluster_embeddings = ClientModel.predict(image_path, True, False)
np.save("cluster_embeddings.npy", cluster_embeddings)

### Checklist
1) Move Laplace Noise into model ✅
2) Write Siamese Noise ✅
    - Add color classification ✅
    - Add item classification 🟡 (Results don't improve and too much storage)
    - For each entry in the embedding, move 'alpha' closer to the cluster ✅
3) Add Distance Metric to Test ✅
4) Split ResNet Loading for size reasons 🟡 (IDK if it worked)
5) Create ReadME of workflow ✅

### Metrics Code

In [33]:
# Epsilon - Strength of Privacy Protection for datasets
# This most likely doesn't apply to us since we are sending one piece of data
# The epsilon values between 0 to 1 are considered highly private. 
# The values between 2 to 10 for epsilon are considered better to have than nothing. 
# And the values above 10 are almost sharing the exact data exposing user privacy.

noise_scale = 0.35
output_sensitivity = 3.277522
epsilon = output_sensitivity / noise_scale
print(epsilon)


9.364348571428572


In [27]:
# Model Size

ClientModel.build((1, 224, 224, 3))
ClientModel.summary()

Model: "client_res_net_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_2 (Functional)        (None, 28, 28, 512)       1460096   
                                                                 
Total params: 1,460,096
Trainable params: 0
Non-trainable params: 1,460,096
_________________________________________________________________


In [14]:
# See model layers
import tensorflow.keras.applications as keras_applications

model = keras_applications.ResNet50(weights='imagenet', include_top=False)
for layer in model.layers:
    print(layer.name, layer.output_shape)

input_3 [(None, None, None, 3)]
conv1_pad (None, None, None, 3)
conv1_conv (None, None, None, 64)
conv1_bn (None, None, None, 64)
conv1_relu (None, None, None, 64)
pool1_pad (None, None, None, 64)
pool1_pool (None, None, None, 64)
conv2_block1_1_conv (None, None, None, 64)
conv2_block1_1_bn (None, None, None, 64)
conv2_block1_1_relu (None, None, None, 64)
conv2_block1_2_conv (None, None, None, 64)
conv2_block1_2_bn (None, None, None, 64)
conv2_block1_2_relu (None, None, None, 64)
conv2_block1_0_conv (None, None, None, 256)
conv2_block1_3_conv (None, None, None, 256)
conv2_block1_0_bn (None, None, None, 256)
conv2_block1_3_bn (None, None, None, 256)
conv2_block1_add (None, None, None, 256)
conv2_block1_out (None, None, None, 256)
conv2_block2_1_conv (None, None, None, 64)
conv2_block2_1_bn (None, None, None, 64)
conv2_block2_1_relu (None, None, None, 64)
conv2_block2_2_conv (None, None, None, 64)
conv2_block2_2_bn (None, None, None, 64)
conv2_block2_2_relu (None, None, None, 64)
conv2_b